###  Identifying all of the fields containing either date or time data.

In [1]:
# Import pandas
import pandas as pd
# reading csv file
df = pd.read_csv("OR Booking.csv")
df.dtypes

HCID                      int64
Pt Age                  float64
Proc Descr Mod           object
Req Proc Tm              object
Pt Priority              object
OR Booking Req DT/Tm     object
Pt Loc                   object
Proc DT                  object
Pt OR Chk In             object
Pt In OR                 object
OR                        int64
Pt Trns                  object
ORR#                      int64
dtype: object

### Data type of each field

In [2]:

df.dtypes

HCID                      int64
Pt Age                  float64
Proc Descr Mod           object
Req Proc Tm              object
Pt Priority              object
OR Booking Req DT/Tm     object
Pt Loc                   object
Proc DT                  object
Pt OR Chk In             object
Pt In OR                 object
OR                        int64
Pt Trns                  object
ORR#                      int64
dtype: object

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1229 entries, 0 to 1228
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   HCID                  1229 non-null   int64  
 1   Pt Age                1164 non-null   float64
 2   Proc Descr Mod        1229 non-null   object 
 3   Req Proc Tm           1229 non-null   object 
 4   Pt Priority           1229 non-null   object 
 5   OR Booking Req DT/Tm  1229 non-null   object 
 6   Pt Loc                1229 non-null   object 
 7   Proc DT               1229 non-null   object 
 8   Pt OR Chk In          1229 non-null   object 
 9   Pt In OR              1229 non-null   object 
 10  OR                    1229 non-null   int64  
 11  Pt Trns               1229 non-null   object 
 12  ORR#                  1229 non-null   int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 124.9+ KB


###  The PT priority column reflects the number of hours before they should be operated based on their condition.  (for example E-06H means the should be operated on with 6 hours after their diagnosis and subsequent booking).

### Creating a new column with a date time for when each patient should have their procedure started by based on the PT priority column compared to the OR Booking Req DT/Tm. 

In [4]:
df['Pt Priority']=  df['Pt Priority'].astype('str').str.extract('(\d+)').astype(int)
df['Pt Priority'] = pd.to_timedelta(df['Pt Priority'], unit='h')
df['OR Booking Req DT/Tm'] = pd.to_datetime(df['OR Booking Req DT/Tm'])
df['Procedure Starts'] = df['OR Booking Req DT/Tm'] + df['Pt Priority'] 
df.head()

,HCID,Pt Age,Proc Descr Mod,Req Proc Tm,Pt Priority,OR Booking Req DT/Tm,Pt Loc,Proc DT,Pt OR Chk In,Pt In OR,OR,Pt Trns,ORR#,Procedure Starts
0,1805294,40.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-12 03:00:00,35,2019-01-12,1000,1025,5,PARU,1065487,2019-01-12 09:00:00
1,2233815,58.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-21 17:12:00,ER,2019-01-21,1827,1905,5,PARU,1065535,2019-01-22 17:12:00
2,1043375,31.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-22 03:30:00,ER,2019-01-22,0750,0800,5,PARU,1065583,2019-01-22 09:30:00
3,1203917,45.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-29 18:12:00,56,2019-01-30,1728,1805,7,PARU,1065615,2019-01-30 18:12:00
4,2616633,46.0,Laparoscopy Appendectomy,,0 days 06:00:00,2019-01-30 05:30:00,35,2019-01-30,0740,0800,10,PARU,1065660,2019-01-30 11:30:00


### In order to start looking at the amount of time patients are waiting before surgery, you will need to compare the OR Booking Req DT/Tm to the actual procedure time.  The OR is a 24 hour operation.

#### Making a choice between the Pt OR Chk In or the Pt In OR time columns as the time to pair with the date for surgery for this new column.



###### A case in point for patient with HCID 1805294 with corresponding scheduled OR Booking Req DT/Tm as  '2019-01-12 3:00' and actual datetime Pt Chk In was '2019-01-12 10:00' and  actual datetime Pt In OR was '2019-01-12 10:25' . The deduction to make from this is that, there was a 7hr delay before patient was checked in to the operation room on the scheduled day of operation and it took 25min after check in for surgery to start which is the the actual procedure time. Hence a correct comparison of datetime between scheduled datetime and actual procedure datetime will be to use the Pt In Or time. 

####  Creating a new column combining the procedure date and either of the Pt OR Check In or the Pt In OR time to a datetime format. 

In [5]:
df['Pt In OR']=[f"{el[:2]}:{el[2:]}" for el in df['Pt In OR']]

In [6]:
df['Proc_DateTime'] = df['Proc DT'] + ' ' + df['Pt In OR']

In [7]:
df['Proc_DateTime'] = pd.to_datetime(df['Proc_DateTime'], errors='coerce')

In [8]:
df.head()

,HCID,Pt Age,Proc Descr Mod,Req Proc Tm,Pt Priority,OR Booking Req DT/Tm,Pt Loc,Proc DT,Pt OR Chk In,Pt In OR,OR,Pt Trns,ORR#,Procedure Starts,Proc_DateTime
0,1805294,40.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-12 03:00:00,35,2019-01-12,1000,10:25,5,PARU,1065487,2019-01-12 09:00:00,2019-01-12 10:25:00
1,2233815,58.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-21 17:12:00,ER,2019-01-21,1827,19:05,5,PARU,1065535,2019-01-22 17:12:00,2019-01-21 19:05:00
2,1043375,31.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-22 03:30:00,ER,2019-01-22,0750,08:00,5,PARU,1065583,2019-01-22 09:30:00,2019-01-22 08:00:00
3,1203917,45.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-29 18:12:00,56,2019-01-30,1728,18:05,7,PARU,1065615,2019-01-30 18:12:00,2019-01-30 18:05:00
4,2616633,46.0,Laparoscopy Appendectomy,,0 days 06:00:00,2019-01-30 05:30:00,35,2019-01-30,0740,08:00,10,PARU,1065660,2019-01-30 11:30:00,2019-01-30 08:00:00


### Analyzing the performance for getting the patients completed in within specified time.

In [9]:
df['Time Difference'] = df['Proc_DateTime'] - df['OR Booking Req DT/Tm']
df.head()

,HCID,Pt Age,Proc Descr Mod,Req Proc Tm,Pt Priority,OR Booking Req DT/Tm,Pt Loc,Proc DT,Pt OR Chk In,Pt In OR,OR,Pt Trns,ORR#,Procedure Starts,Proc_DateTime,Time Difference
0,1805294,40.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-12 03:00:00,35,2019-01-12,1000,10:25,5,PARU,1065487,2019-01-12 09:00:00,2019-01-12 10:25:00,0 days 07:25:00
1,2233815,58.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-21 17:12:00,ER,2019-01-21,1827,19:05,5,PARU,1065535,2019-01-22 17:12:00,2019-01-21 19:05:00,0 days 01:53:00
2,1043375,31.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-22 03:30:00,ER,2019-01-22,0750,08:00,5,PARU,1065583,2019-01-22 09:30:00,2019-01-22 08:00:00,0 days 04:30:00
3,1203917,45.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-29 18:12:00,56,2019-01-30,1728,18:05,7,PARU,1065615,2019-01-30 18:12:00,2019-01-30 18:05:00,0 days 23:53:00
4,2616633,46.0,Laparoscopy Appendectomy,,0 days 06:00:00,2019-01-30 05:30:00,35,2019-01-30,0740,08:00,10,PARU,1065660,2019-01-30 11:30:00,2019-01-30 08:00:00,0 days 02:30:00


#### Creating another new column to measure if the procedure was started on time

In [10]:
df['Procedure Status'] = None
df.loc[df['Proc_DateTime'] < df['Procedure Starts'], 'Procedure Status'] = 'Early'
df.loc[df['Proc_DateTime'] == df['Procedure Starts'], 'Procedure Status'] = 'On Time'
df.loc[df['Proc_DateTime'] > df['Procedure Starts'], 'Procedure Status'] = 'Delayed'
df

,HCID,Pt Age,Proc Descr Mod,Req Proc Tm,Pt Priority,OR Booking Req DT/Tm,Pt Loc,Proc DT,Pt OR Chk In,Pt In OR,OR,Pt Trns,ORR#,Procedure Starts,Proc_DateTime,Time Difference,Procedure Status
0,1805294,40.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-12 03:00:00,35,2019-01-12,1000,10:25,5,PARU,1065487,2019-01-12 09:00:00,2019-01-12 10:25:00,0 days 07:25:00,Delayed
1,2233815,58.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-21 17:12:00,ER,2019-01-21,1827,19:05,5,PARU,1065535,2019-01-22 17:12:00,2019-01-21 19:05:00,0 days 01:53:00,Early
2,1043375,31.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-22 03:30:00,ER,2019-01-22,0750,08:00,5,PARU,1065583,2019-01-22 09:30:00,2019-01-22 08:00:00,0 days 04:30:00,Early
3,1203917,45.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-29 18:12:00,56,2019-01-30,1728,18:05,7,PARU,1065615,2019-01-30 18:12:00,2019-01-30 18:05:00,0 days 23:53:00,Early
4,2616633,46.0,Laparoscopy Appendectomy,,0 days 06:00:00,2019-01-30 05:30:00,35,2019-01-30,0740,08:00,10,PARU,1065660,2019-01-30 11:30:00,2019-01-30 08:00:00,0 days 02:30:00,Early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,1547397,25.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2021-01-19 06:06:00,ER,2021-01-19,1400,14:41,9,PARU,1100943,2021-01-19 12:06:00,2021-01-19 14:41:00,0 days 08:35:00,Delayed
1225,2250377,54.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2021-02-05 20:15:00,35,2021-02-06,1557,16:40,9,PARU,1100996,2021-02-06 20:15:00,2021-02-06 16:40:00,0 days 20:25:00,Early
1226,2803120,18.0,Laparoscopy Cholecystectomy,0060,1 days 12:00:00,2021-02-07 14:32:00,64,2021-02-08,1541,16:08,10,PARU,1101049,2021-02-09 02:32:00,2021-02-08 16:08:00,1 days 01:36:00,Early
1227,1602995,25.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2021-02-08 05:56:00,ER,2021-02-08,0908,09:50,4,,1101074,2021-02-08 11:56:00,2021-02-08 09:50:00,0 days 03:54:00,Early


#### Creating another new column to measure the difference in time between the time the procedure was due and when it was started 

In [11]:
df['measure the difference'] = df['Proc_DateTime'] -df['Procedure Starts']
df

,HCID,Pt Age,Proc Descr Mod,Req Proc Tm,Pt Priority,OR Booking Req DT/Tm,Pt Loc,Proc DT,Pt OR Chk In,Pt In OR,OR,Pt Trns,ORR#,Procedure Starts,Proc_DateTime,Time Difference,Procedure Status,measure the difference
0,1805294,40.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-12 03:00:00,35,2019-01-12,1000,10:25,5,PARU,1065487,2019-01-12 09:00:00,2019-01-12 10:25:00,0 days 07:25:00,Delayed,0 days 01:25:00
1,2233815,58.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-21 17:12:00,ER,2019-01-21,1827,19:05,5,PARU,1065535,2019-01-22 17:12:00,2019-01-21 19:05:00,0 days 01:53:00,Early,-1 days +01:53:00
2,1043375,31.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2019-01-22 03:30:00,ER,2019-01-22,0750,08:00,5,PARU,1065583,2019-01-22 09:30:00,2019-01-22 08:00:00,0 days 04:30:00,Early,-1 days +22:30:00
3,1203917,45.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2019-01-29 18:12:00,56,2019-01-30,1728,18:05,7,PARU,1065615,2019-01-30 18:12:00,2019-01-30 18:05:00,0 days 23:53:00,Early,-1 days +23:53:00
4,2616633,46.0,Laparoscopy Appendectomy,,0 days 06:00:00,2019-01-30 05:30:00,35,2019-01-30,0740,08:00,10,PARU,1065660,2019-01-30 11:30:00,2019-01-30 08:00:00,0 days 02:30:00,Early,-1 days +20:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,1547397,25.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2021-01-19 06:06:00,ER,2021-01-19,1400,14:41,9,PARU,1100943,2021-01-19 12:06:00,2021-01-19 14:41:00,0 days 08:35:00,Delayed,0 days 02:35:00
1225,2250377,54.0,Laparoscopy Cholecystectomy,0060,1 days 00:00:00,2021-02-05 20:15:00,35,2021-02-06,1557,16:40,9,PARU,1100996,2021-02-06 20:15:00,2021-02-06 16:40:00,0 days 20:25:00,Early,-1 days +20:25:00
1226,2803120,18.0,Laparoscopy Cholecystectomy,0060,1 days 12:00:00,2021-02-07 14:32:00,64,2021-02-08,1541,16:08,10,PARU,1101049,2021-02-09 02:32:00,2021-02-08 16:08:00,1 days 01:36:00,Early,-1 days +13:36:00
1227,1602995,25.0,Laparoscopy Appendectomy,0060,0 days 06:00:00,2021-02-08 05:56:00,ER,2021-02-08,0908,09:50,4,,1101074,2021-02-08 11:56:00,2021-02-08 09:50:00,0 days 03:54:00,Early,-1 days +21:54:00
